# Working with large and small integers

Python does not make a distinction between integers, bytes and long integers; signed or unsigned integers. <br>
Python handles integers of all sizes in a uniform way.

Internally, Python has two representations for numbers. <br>
For smallish numbers, Python will generally use 4-byte or 8-byte integer values. <br>
For numbers over **sys.maxsize**, Python switches to internally representing integer numbers as sequences of digits.<br> 
Digit, in this case, often means a 30-bit value.


In [1]:
import math
math.factorial(52)

80658175170943878571660636856403766975289505440883277824000000000000

The first parts of our calculation of 52! (from 52 × 51 × 50 × ... down to about 42) could be performed entirely using the smallish integers. <br> After that, the rest of the calculation had to switch to largish integers.

We don't see the switch; we only see the results. The conversion between these two is seamless and automatic.

In [2]:
import sys
import math
math.log(sys.maxsize, 2)

63.0

In [3]:
sys.int_info

sys.int_info(bits_per_digit=30, sizeof_digit=4)

The **sys.maxsize** value is the largest of the small integer values. <br>
We computed the log to base 2 to find out how many bits are required for this number.

This tells us that our Python uses 63-bit values for small integers. <br> 
The range of smallish integers is from $-2^{63}$ ... $2^{63} - 1$. <br>
Outside this range, largish integers are used.

The values in **sys.int_info** tell us that large integers are a sequence of 30-bit digits.<br>
Each of these digits occupies 4 bytes.

For details on integer processing, see https://www.python.org/dev/peps/pep-0237/

# Choosing between float, decimal and fraction

## Doing currency calculations

In [4]:
from decimal import Decimal
tax_rate = Decimal('7.25')/Decimal(100)
purchase_amount = Decimal('2.95')
tax_rate * purchase_amount

Decimal('0.213875')

In [5]:
penny = Decimal('0.01')
total_amount = purchase_amount + tax_rate * purchase_amount
total_amount.quantize(penny)

Decimal('3.16')

In [11]:
import decimal
total_amount.quantize(penny, decimal.ROUND_UP)

Decimal('3.17')

## Fraction Calcuations

In [12]:
from fractions import Fraction
sugar_cups = Fraction('2.5')
scale_factor = Fraction(5/8)
sugar_cups * scale_factor

Fraction(25, 16)

We created one fraction from a string and the second from a floating-point calculation. <br>
Because the denominator is a power of 2, this works out exactly.

In [13]:
Fraction(24, 16)

Fraction(3, 2)

## Floating Point Approximations

In [14]:
(19/155)*(155/19)

0.9999999999999999

All float values and calculations are an approximation.<br>
We often need to round values to make them look sensible.<br>
Mathematically, the value should be 1. <br>
Round to 3 digits:

In [15]:
answer = (19/155)*(155/19)
round(answer, 3)

1.0

In [16]:
1 - answer

1.1102230246251565e-16

For most floating-point errors, this is the typical value - about $10^{-16}$.

**Don't compare floating-point values for exact equality.**

## Converting numbers from one type into another

Convert a Decimal and Fraction value into float:

In [17]:
float(total_amount)

3.163875

In [18]:
float(sugar_cups * scale_factor)

1.5625

Converting float into Decimal or Fraction results in complicated values.

In [19]:
Fraction(19/155)

Fraction(8832866365939553, 72057594037927936)

In [20]:
Decimal(19/155)

Decimal('0.12258064516129031640279123394066118635237216949462890625')

Python is adept at converting numbers between the various types:<br>
1. We can mix int and float values; the integers will be promoted to floating-point.
2. We can mix int and Fraction and the results will be a Fraction object. 
3. We can mix int and Decimal. 
4. We cannot casually mix Decimal with float or Fraction; we need to provide explicit conversions in that case.

The Python syntax allows us to write numbers as decimal values; however, that's not how they're processed internally.

We can write a value like this in Python, using ordinary base-10 values:

In [21]:
8.066e+67

8.066e+67

The actual value used internally will involve a binary approximation of the decimal value.<br>
The internal value for this example is this:

In [22]:
(6737037547376141/(2**53))*(2**226)

8.066e+67

The numerator is a big number, 6737037547376141. The denominator is always $2^{53}$. <br>
Since the denominator is fixed, the resulting fraction can only have 53 meaningful bits of data.<br>
This is why values can get truncated. <br>
This leads to tiny discrepancies between our idealized abstraction and actual numbers. <br>
The exponent (2226) is required to scale the fraction up to the proper range.


Mathematically, $\frac{6737037547376141 \times 2^{226}}{2^{53}}$

In [23]:
import math
math.frexp(8.066E+67)

(0.7479614202861186, 226)

The internal details of a number. The two parts are called the **mantissa** (or significand) and the **exponent**.<br> If we multiply the mantissa by $2^{53}$, we always get a whole number, which is the numerator of the binary fraction.